# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [47]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
from nltk import word_tokenize, pos_tag, ne_chunk, WordNetLemmatizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
# import category_encoders as ce

In [25]:
# load data from database
engine = create_engine('sqlite:///../data/Disaster_Response.db')
# df = pd.read_sql("SELECT * FROM Disaster_Response", engine)
df = pd.read_sql_table("Disaster_Response", engine)
X = df.loc[:,'message':'genre']
Y = df.loc[:,'related':]
X.shape, Y.shape

((26180, 3), (26180, 36))

### 2. Write a tokenization function to process your text data

In [3]:
# regex pattern for detecting a url
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex,text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url,'urlplaceholder')

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [4]:
# test the tokenizer
text=X.original[0]
print(tokenize(text))

['un', 'front', 'froid', 'se', 'retrouve', 'sur', 'cuba', 'ce', 'matin', '.', 'il', 'pourrait', 'traverser', 'haiti', 'demain', '.', 'des', 'averses', 'de', 'pluie', 'isolee', 'sont', 'encore', 'prevue', 'sur', 'notre', 'region', 'ce', 'soi']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [34]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier())
])

In [29]:
# # try to preprocessing pipelines for both text and categorical data.
# # Debugging needed

# text_features = ['message']
# text_transformer = Pipeline(steps=[
#     ('vect', CountVectorizer(tokenizer=tokenize)),
#     ('tfidf', TfidfTransformer())])

# categorical_features = ['genre']
# categorical_transformer = Pipeline(steps=[
#     # the 'genre' column does not actually have missing values
#     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#     ('onehot', LabelEncoder(handle_unknown='ignore'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('text', text_transformer, text_features),
#         ('cate', categorical_transformer, categorical_features)])

# # Append classifier to preprocessing pipeline.
# # Now we have a full prediction pipeline.
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', RandomForestClassifier())])

# X_train, X_test, y_train, y_test = train_test_split(X[['message','genre']], Y, test_size=0.2)

# # clf.fit(X_train, y_train)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [35]:
# train text split
X_train, X_test, y_train, y_test = train_test_split(X.message, Y, random_state=1234)

# train the pipeline
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [50]:
for i, col in enumerate(Y.columns):
    print('\033[1m'+col+'\033[0m', '\n', metrics.classification_report(y_test.iloc[:,i], y_pred[:,i]))

related 
               precision    recall  f1-score   support

           0       0.56      0.42      0.48      1527
           1       0.83      0.90      0.86      4978
           2       0.42      0.28      0.33        40

    accuracy                           0.78      6545
   macro avg       0.61      0.53      0.56      6545
weighted avg       0.77      0.78      0.77      6545

request 
               precision    recall  f1-score   support

           0       0.89      0.99      0.93      5465
           1       0.86      0.36      0.51      1080

    accuracy                           0.88      6545
   macro avg       0.87      0.67      0.72      6545
weighted avg       0.88      0.88      0.86      6545

offer 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6511
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6545
   macro avg       0.50      0.50      0.50   

hospitals 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6477
           1       0.00      0.00      0.00        68

    accuracy                           0.99      6545
   macro avg       0.49      0.50      0.50      6545
weighted avg       0.98      0.99      0.98      6545

shops 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6519
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.99      1.00      0.99      6545

aid_centers 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6468
           1       0.00      0.00      0.00        77

    accuracy                           0.99      6545
   macro avg       0.49      0.50      0.50      6545
weighted avg       0.98      0.99      0

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.